In [5]:
# !pip install openai
# !pip install tiktoken

In [118]:
import openai
import pandas as pd
import numpy as np
import tiktoken
import datetime
from IPython.display import Image

In [7]:
# f = open("/content/sample_data/OPENAI_API_KEY.txt", "r")
f = open("OPENAI_API_KEY.txt", "r")
openai_api_key = f.read()
# print(openai_api_key)
client = openai.OpenAI(api_key=openai_api_key)

In [8]:
llm_model = "gpt-3.5-turbo"

In [9]:
def get_completion_and_token_count(prompt,
                                   model=llm_model,
                                   temperature=0,
                                   max_tokens=500):
    messages = [{"role": "user", "content": prompt+' output in JSON format'}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,        # messages = [{"role": "user", "content": prompt}], a list of messages comprising the conversation so far.
        temperature=temperature,
        max_tokens=max_tokens,    # The maximum number of tokens that can be 'generated' by the chat completion.
    )

    content = response.choices[0].message.content

    token_dict = {
        'prompt_tokens':response.usage.prompt_tokens, # no. of input tokens counted by OpenAI API
        'completion_tokens':response.usage.completion_tokens,
        'total_tokens':response.usage.total_tokens,
    }
    print(response)
    print(content)
    print(token_dict)

    return response, content, token_dict

In [10]:
response, content, token_dict = get_completion_and_token_count("What is 1+1?")

ChatCompletion(id='chatcmpl-936xiynhdgEgMfYCEjLNdQJB3uRWM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "result": 2\n}', role='assistant', function_call=None, tool_calls=None))], created=1710528774, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f2ebda25a', usage=CompletionUsage(completion_tokens=9, prompt_tokens=18, total_tokens=27))
{
  "result": 2
}
{'prompt_tokens': 18, 'completion_tokens': 9, 'total_tokens': 27}


In [11]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
encoding.encode("What is 1+1?") # no. of input tokens counted by OpenAI API

[3923, 374, 220, 16, 10, 16, 30]

# ***Chat API : LangChain***

In [ ]:
# !pip install langchain --user

In [21]:
import langchain

# Model

In [22]:
from langchain.chat_models import ChatOpenAI

In [23]:
import os
os.environ['OPENAI_API_KEY'] = openai_api_key # MUST
# To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)

C:\Users\amit.tewari\AppData\Roaming\Python\Python38\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Prompt template - Input into the model

In [24]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [25]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(template_string)

In [26]:
print(prompt_template.messages[0].prompt)
print(prompt_template.messages[0].prompt.input_variables)

input_variables=['style', 'text'] template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
['style', 'text']


In [27]:
customer_email = """
Hi how are you doing
"""
customer_style = """Spanish \
in a calm and respectful tone
"""
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [28]:
print(type(customer_messages))
print(customer_messages)
print(type(customer_messages[0]))
print(customer_messages[0])

<class 'list'>
[HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is Spanish in a calm and respectful tone\n. text: ```\nHi how are you doing\n```\n')]
<class 'langchain_core.messages.human.HumanMessage'>
content='Translate the text that is delimited by triple backticks into a style that is Spanish in a calm and respectful tone\n. text: ```\nHi how are you doing\n```\n'


In [29]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

C:\Users\amit.tewari\AppData\Roaming\Python\Python38\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [30]:
print(customer_response.content)

Hola, ¿cómo estás?


# Parse the LLM output 

In [31]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [32]:
email_body_schema = ResponseSchema(name="email_body",
                                      description="Extract email body")

email_from_schema = ResponseSchema(name="email_sender",
                                      description="Extract sender email")

email_from_name_schema = ResponseSchema(name="email_sender_name",
                                      description="Extract sender name")

email_from_designation_schema = ResponseSchema(name="email_sender_designation",
                                      description="Extract sender designation")

email_from_organization_schema = ResponseSchema(name="email_sender_organization",
                                      description="Extract sender orgaization")

email_date_schema = ResponseSchema(name="email_date",
                                       description="Extract email date")

email_to_schema = ResponseSchema(name="email_recipients",
                                      description="Extract comma separated email recipients")

response_schemas = [email_from_schema,
                    email_from_name_schema,
                    email_from_designation_schema,
                    email_from_organization_schema,
                    email_to_schema,
                    email_date_schema,
                    email_body_schema]

In [33]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [34]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"email_sender": string  // Extract sender email
	"email_sender_name": string  // Extract sender name
	"email_sender_designation": string  // Extract sender designation
	"email_sender_organization": string  // Extract sender orgaization
	"email_recipients": string  // Extract comma separated email recipients
	"email_date": string  // Extract email date
	"email_body": string  // Extract email body
}
```


In [35]:
email = "From: Doe, John <JohnDoe@xyz.com> \
Sent: Wednesday, March 15, 2023 11:21 PM \
To: Hollester, Mike <@MHollester.com>; Parrera, Nicky <NParrera@xyz.com> \
Cc: Bush, Johnny <JBush@xyz.com>; Herding, Bill <BHerding@xyz.com> \
Subject: RE: Interested in AI product  \
I hope this email finds you well. \
I am reaching out to inquire about your company's AI offerings and services. \
Could you please provide more information regarding your capabilities and solutions in this field? \
Shawn Ponting \
Senior Product Manager \
Great Corporation"

In [36]:
review_template_2 = """\
For the following text, extract the following information:

email_body: Extract email body If this information is not found,output Unknown Email Body
email_sender: Extract sender email If this information is not found, output Unknown Sender.
email_sender_designation: Extract sender designation If this information is not found, output Unknown Sender Designation.
email_sender_organization: Extract sender organization If this information is not found, output Unknown Sender Organization.
email_date: Extract email date If this information is not found, output Unknown Date.
email_recipients: Extract comma separated email recipients If this information is not found, output Unknown Recipients.

text: {text}

{format_instructions}
"""

In [37]:
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=email,
                                format_instructions=format_instructions)

In [38]:
print(messages[0].content)

For the following text, extract the following information:

email_body: Extract email body If this information is not found,output Unknown Email Body
email_sender: Extract sender email If this information is not found, output Unknown Sender.
email_sender_designation: Extract sender designation If this information is not found, output Unknown Sender Designation.
email_sender_organization: Extract sender organization If this information is not found, output Unknown Sender Organization.
email_date: Extract email date If this information is not found, output Unknown Date.
email_recipients: Extract comma separated email recipients If this information is not found, output Unknown Recipients.

text: From: Doe, John <JohnDoe@xyz.com> Sent: Wednesday, March 15, 2023 11:21 PM To: Hollester, Mike <@MHollester.com>; Parrera, Nicky <NParrera@xyz.com> Cc: Bush, Johnny <JBush@xyz.com>; Herding, Bill <BHerding@xyz.com> Subject: RE: Interested in AI product  I hope this email finds you well. I am reach

In [39]:
response = chat(messages)

In [40]:
print(response.content)

```json
{
	"email_sender": "JohnDoe@xyz.com",
	"email_sender_name": "Doe, John",
	"email_sender_designation": "Senior Product Manager",
	"email_sender_organization": "Great Corporation",
	"email_recipients": "Hollester, Mike <@MHollester.com>, Parrera, Nicky <NParrera@xyz.com>, Bush, Johnny <JBush@xyz.com>, Herding, Bill <BHerding@xyz.com>",
	"email_date": "Wednesday, March 15, 2023 11:21 PM",
	"email_body": "I hope this email finds you well. I am reaching out to inquire about your company's AI offerings and services. Could you please provide more information regarding your capabilities and solutions in this field?"
}
```


In [41]:
output_dict = output_parser.parse(response.content)

In [42]:
output_dict

{'email_sender': 'JohnDoe@xyz.com',
 'email_sender_name': 'Doe, John',
 'email_sender_designation': 'Senior Product Manager',
 'email_sender_organization': 'Great Corporation',
 'email_recipients': 'Hollester, Mike <@MHollester.com>, Parrera, Nicky <NParrera@xyz.com>, Bush, Johnny <JBush@xyz.com>, Herding, Bill <BHerding@xyz.com>',
 'email_date': 'Wednesday, March 15, 2023 11:21 PM',
 'email_body': "I hope this email finds you well. I am reaching out to inquire about your company's AI offerings and services. Could you please provide more information regarding your capabilities and solutions in this field?"}

# LangChain: Memory

### Memory of history in a chat keeps on appending by Langchain!

In [68]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [69]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [70]:
conversation.predict(input="Hi, this is a LangChain tutorial to explain Memory for chat application")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, this is a LangChain tutorial to explain Memory for chat application
AI:

> Finished chain.


"Hello! I'm excited to help you understand Memory for chat applications. Memory in this context refers to the ability of the chat application to remember past interactions and information shared between users. This can include things like chat history, user preferences, and personalized recommendations based on previous conversations. Is there a specific aspect of Memory for chat applications that you would like me to explain further?"

In [71]:
conversation.predict(input="I will use simple examples to explain Langchain Memory ")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, this is a LangChain tutorial to explain Memory for chat application
AI: Hello! I'm excited to help you understand Memory for chat applications. Memory in this context refers to the ability of the chat application to remember past interactions and information shared between users. This can include things like chat history, user preferences, and personalized recommendations based on previous conversations. Is there a specific aspect of Memory for chat applications that you would like me to explain further?
Human: I will use simple examples to explain Langchain Memory 
AI:

> Finished chain.


"That sounds like a great approach! Using simple examples can really help clarify how Memory works in LangChain. Feel free to provide any examples or scenarios you have in mind, and I'll do my best to explain how Memory would be utilized in those situations. Let's dive into it!"

In [72]:
conversation.predict(input="What are we going to learn in this tutorial?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, this is a LangChain tutorial to explain Memory for chat application
AI: Hello! I'm excited to help you understand Memory for chat applications. Memory in this context refers to the ability of the chat application to remember past interactions and information shared between users. This can include things like chat history, user preferences, and personalized recommendations based on previous conversations. Is there a specific aspect of Memory for chat applications that you would like me to explain further?
Human: I will use simple examples to explain Langchain Memory 
AI: That sounds like a great approach! Using simple examples can really help clarify how Memo

"In this tutorial, we will learn about how Memory is used in LangChain to enhance chat applications. We will explore how Memory can store and retrieve past interactions, user preferences, and personalized recommendations. Additionally, we will discuss how Memory can improve user experience and make chat applications more engaging and efficient. By the end of this tutorial, you will have a better understanding of how Memory works in LangChain and how it can benefit chat applications. Let's get started!"

In [73]:
print(memory.buffer) # History

Human: Hi, this is a LangChain tutorial to explain Memory for chat application
AI: Hello! I'm excited to help you understand Memory for chat applications. Memory in this context refers to the ability of the chat application to remember past interactions and information shared between users. This can include things like chat history, user preferences, and personalized recommendations based on previous conversations. Is there a specific aspect of Memory for chat applications that you would like me to explain further?
Human: I will use simple examples to explain Langchain Memory 
AI: That sounds like a great approach! Using simple examples can really help clarify how Memory works in LangChain. Feel free to provide any examples or scenarios you have in mind, and I'll do my best to explain how Memory would be utilized in those situations. Let's dive into it!
Human: What are we going to learn in this tutorial?
AI: In this tutorial, we will learn about how Memory is used in LangChain to enhan

In [88]:
# Provide additional context 
today_date = str(datetime.date.today())
memory.save_context({"input": "What is the date today?" }, 
                    {"output": today_date}) 

In [89]:
memory.load_memory_variables({})

{'history': "Human: Hi, this is a LangChain tutorial to explain Memory for chat application\nAI: Hello! I'm excited to help you understand Memory for chat applications. Memory in this context refers to the ability of the chat application to remember past interactions and information shared between users. This can include things like chat history, user preferences, and personalized recommendations based on previous conversations. Is there a specific aspect of Memory for chat applications that you would like me to explain further?\nHuman: I will use simple examples to explain Langchain Memory \nAI: That sounds like a great approach! Using simple examples can really help clarify how Memory works in LangChain. Feel free to provide any examples or scenarios you have in mind, and I'll do my best to explain how Memory would be utilized in those situations. Let's dive into it!\nHuman: What are we going to learn in this tutorial?\nAI: In this tutorial, we will learn about how Memory is used in 

In [96]:
# Remember just 'LAST K converstaions' instead of entire history to avoid sending entire conversation to OpenAI API
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1)               

In [97]:
# Provide additional context 
today_date = str(datetime.date.today())
memory.save_context({"input": "What is your City of residenc?" }, 
                    {"output": "Delhi"}) 
memory.save_context({"input": "What is your Country of residence?" }, 
                    {"output": "India"}) 

In [98]:
memory.load_memory_variables({})

{'history': 'Human: What is your Country of residence?\nAI: India'}

In [99]:
# llm = ChatOpenAI(temperature=0.0, model=llm_model)
# memory =  ConversationBufferWindowMemory(k=1)  # usage
# conversation = ConversationChain(
#     llm=llm, 
#     memory = memory,
#     verbose=True
# )

In [107]:
# Limit the # of tokens to send instead of entire history
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [108]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20)
memory.save_context({"input": "What is your City of residence?" }, 
                    {"output": "Delhi"}) 
memory.save_context({"input": "What is your Country of residence?" }, 
                    {"output": "India"})
memory.save_context({"input": "Which city is your office located?" }, 
                    {"output": "New York"})

In [109]:
memory.load_memory_variables({})

{'history': 'AI: New York'}

In [113]:
# Instead of limiting fixed # of token, lets use LLM to write the summary of conversation so far and remem the summary
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=30) 
# entire hostory so far will be summarized in max 30 token 
memory.save_context({"input": "What is your City of residence?" }, 
                    {"output": "Delhi"}) 
memory.save_context({"input": "What is your Country of residence?" }, 
                    {"output": "India"})
memory.save_context({"input": "Which city is your office located?" }, 
                    {"output": "New York"})
memory.save_context({"input": "What is the date today?" }, 
                    {"output": today_date}) 

In [114]:
memory.load_memory_variables({})

{'history': 'System: The human asks the AI\'s City of residence, to which the AI responds "Delhi." When asked for the AI\'s Country of residence, the AI replies "India." The human then inquires about the city where the AI\'s office is located, and the AI answers "New York."\nHuman: What is the date today?\nAI: 2024-03-16'}

In [115]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [117]:
# Q/A or chat using history/memory 
conversation.predict(input="What is the office location city?") 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human asks the AI's City of residence, to which the AI responds "Delhi." When asked for the AI's Country of residence, the AI replies "India." The human then inquires about the city where the AI's office is located, and the AI answers "New York." When asked about the date, the AI responds "2024-03-16."
Human: What is the office location?
AI: The office location is in Manhattan, New York City.
Human: What is the office location city?
AI:

> Finished chain.


'The office location city is New York City.'

In [119]:
Image(filename='img1.png') 

FileNotFoundError: [Errno 2] No such file or directory: 'img1.png'

# Chains